In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 4 – Pivot Tables, Permutation Testing, and Missing Values

## DSC 80, Spring 2022

### Due Date: Monday, April 25th at 11:59 PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying `lab.py` file will be tested (a la DSC 20),
2. The notebook may be graded (if it contains free response questions or asks you to draw plots).

<span style='color:red'><b>Note: For Lab 4 only, there are no hidden tests!</b></span> The tests you see when you run `grader.check` are the final tests that will determine your grade. In addition, when you submit Lab 4 to Gradescope you will see your final score on the assignment right away. (This is because the lab is due very close to the Midterm Exam.)

**Do not change the function names in the `*lab.py` file!**
- The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name.
- If you changed something you weren't supposed to, just use git to revert! Ask us if you need help with this, or google around for `git revert`.

**Tips for working in the notebook**:
- The notebooks serve to present the questions and give you a place to present your results for later review.
- The notebooks in *lab assignments* are not graded (only the `lab.py` file is submitted and graded).
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file.

**Tips for developing in the `lab.py` file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional helper functions to solve the lab! 
- Always document your code!

### Importing code from `lab.py`

* We import our `lab.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lab import *

In [ ]:
import pandas as pd
import numpy as np
import io
import os
from datetime import time

<span style='color:red'><b>New: Check <a href=https://campuswire.com/c/G325FA25B/feed/754>this</a> Campuswire post for any clarifications each time you start working on the lab.</b></span>

### Question 0 – Mid-Quarter Survey 🙋

Course staff, along with the Data Science Student Representatives, have put together a mid-quarter survey that will allow you to share feedback on your experience in DSC 80 so far. It is **entirely anonymous**, so we encourage you to be honest.

<br>

<center><b><a href=https://docs.google.com/forms/d/e/1FAIpQLSd9k90fGqPKDRAnHjFBEx5kak_VtvYN5Fq5uPv9jyqrryaKeA/viewform>Click here to access the mid-quarter survey.</a></b></center>

<br>

We'd like to have as many students as possible in the class fill out the survey. **As such, if 80% of the class fills out this survey before the Midterm Exam, then everyone will earn an extra point on the Midterm Exam.** The survey will close before the Midterm Exam. 

We really appreciate your feedback, thanks! 😊

## Part 1: Time Series Data

Imagine that you own an online store and you'd like to monitor the visits to your site. You've collected some data that you store in `data/login_table.csv`. It contains the information about different login dates and times for different users. Some users are unique, some visited your store multiple times.

You need to answer a few questions below in order to understand the login patters of your users.

### Question 1 – Passwords 🔑

Write a function `latest_login` which takes in a DataFrame like `login` and outputs a DataFrame indexed by `'Login Id'`, of the login that occurs at the latest time-of-day for each user. Latest time-of-day is as it says: the latest time, **regardless of how recent the day is**. The DataFrame should have just one column, named `'Time'`.

For example, if a user always logs in once per day at noon, but her most recent log in happened to be at 8:00PM, then her latest log-in time becomes 8:00PM. Note that the values in your returned DataFrame should only include times, not dates.

***Note:*** You do not need to use Python's `datetime` module – instead, use the built-in `pandas` methods for working with times that we introduced in [Lecture 9](https://dsc80.com/resources/lectures/lec09/lec09.html) (though you will need to do a bit more research to fully answer the question). Do not use a `for`-loop.

In [ ]:
# don't change this cell -- it is needed for the tests to work
fp = os.path.join('data', 'login_table.csv')
login = pd.read_csv(fp)
q1_result = latest_login(login)

In [ ]:
grader.check("q1")

### Question 2 – Return Users 🔁

As a site owner, you would like to see how often users return to your site. You've noticed that there are users who have several logins and users who logged in only once. Of those users that logged in more than once, you are interested in finding the shortest amount of time elapsed between two consecutive logins for each of these users.

Write a function `smallest_elapsed` which takes in a DataFrame like `login` and outputs a DataFrame, indexed by `'Login ID'`, containing the shortest time elapsed for each user. Any users who haven't logged in more than once should not be included in the output. Your DataFrame should only have a single column, named `'Time'`.

***Hint:*** You need not do any querying for this question. There is a built-in `numpy`/`pandas` method that you can use to find the smallest elapsed time for each user.

In [ ]:
# don't change this cell -- it is needed for the tests to work
fp = os.path.join('data', 'login_table.csv')
login = pd.read_csv(fp)
q2_result = smallest_elapsed(login)

In [ ]:
grader.check("q2")

## Part 2: Pivot Tables

### Question 3 – Summarizing Sales 💰

Recall from [Lecture 8](https://dsc80.com/resources/lectures/lec08/lec08.html), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, and are asked to solve a few simple problems using the `pivot_table` method.  

We have provided the outline for your DataFrames, but yours may have a different number of rows and columns and different values.

#### `total_seller`

Write a function `total_seller` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each seller, indexed by `'Name'`. There should not be any `NaN`s.

***Note:*** You may be able to implement `total_seller` without using `pivot_table`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Total</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Write a function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each seller, indexed by `'Product'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Name</th>
      <th>Jones</th>
      <th>Smith</th>
      <th>Trump</th>
    </tr>
    <tr>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>book</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Write a function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Date</th>
      <th>01.01.2012</th>
      <th>02.20.2013</th>
      <th>02.25.2015</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Jones</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Write a function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise, product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
      <th>March</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">Jones</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">Smith</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>


***Note:*** [Here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.

In [ ]:
# don't change this cell -- it is needed for the tests to work
fp = os.path.join('data', 'sales.csv')
sales = pd.read_csv(fp)
q3_total_seller_out = total_seller(sales)
q3_product_name_out = product_name(sales)
q3_product_count_out = count_product(sales)
q3_total_by_month_out = total_by_month(sales)

In [ ]:
grader.check("q3")

## Part 3: Permutation Testing

[Skittles](https://en.wikipedia.org/wiki/Skittles_(confectionery)) 🍬 are made in two locations in the United States: Yorkville, Illinois and Waco, Texas. In these factories, Skittles of different colors are made separately by different machines and combined/packaged into bags for sale. The **tab-separated file** `data/skittles.tsv` contains the contents of 468 bags of Skittles.

Throughout this question, we will compare the color distribution of Skittles between bags made in the Yorkville factory and bags made in the Waco factory. Most people have preferences for their favorite flavor and there is a surprising amount of variation among the distribution of flavors in each bag.

Look at the variation by bag in the dataset below:

In [ ]:
skittles_fp = os.path.join('data', 'skittles.tsv')
skittles = pd.read_csv(skittles_fp, sep='\t')
skittles.head()

In [ ]:
skittles.shape

### Question 4 – Orange Skittles 🟠

First, you will investigate if the machine that mixes together the Skittles of different colors might favor one color over another. Use a permutation test to assess whether, on average, bags made in Yorkville have the same number of orange skittles as bags made in Waco. Do this by implementing the functions described below.

<br>

#### `diff_of_means`

Create a function `diff_of_means` that takes in a DataFrame like `skittles` and returns the **absolute difference** between the **mean** number of orange Skittles per bag from Yorkville and the **mean** number of orange Skittles per bag from Waco.

<br>

#### `simulate_null`

Create a function `simulate_null` that takes in a DataFrame like `skittles` and returns one simulated instance of the test statistic under the null hypothesis. Note that this will involve shuffling!

<br>

#### `pval_color`

Create a function `pval_color` that takes in a DataFrame like `skittles` and calculates the p-value for the permutation test using 1000 trials.

<br>

Plot the observed statistic, along with the histogram for the simulated distribution, to check your work.

***Note:*** In all functions, the default argument for `col` is `'orange'`. Your functions should still work for any color so that you can call it in later questions.

In [ ]:
# don't change this cell -- it is needed for the tests to work
skittles_fp = os.path.join('data', 'skittles.tsv')
skittles = pd.read_csv(skittles_fp, sep='\\t', engine='python')
q4_diff_of_means_out = diff_of_means(skittles)
q4_simulate_null_out = simulate_null(skittles)
q4_many_diffs = np.array([simulate_null(skittles) for _ in range(100)])
q4_pval_out = pval_color(skittles)

In [ ]:
grader.check("q4")

### Question 5 – Generalizing to all colors 🔴🟠🟡🟢🟣

While your `pval_color` function used a default color of `'orange'`, it should also work for all other colors of Skittles, meaning you can run the same permutation test from Question 4 on all colors of Skittles. Call `pval_color` on all colors of Skittles to find which colors differ the most between the two locations on average. 

Then, create a function `ordered_colors` that returns a list of five ordered pairs, each of the form `('color', p_value)`. For example, your list might look like `[('pink', 0.000), ('brown', 0.025), ...]`. 

The list should be **hard-coded**, meaning that you should run your permutation tests in your notebook, not in your `.py` file. The list should also be sorted in **increasing order of p-value**. Make sure your p-values are rounded to **3 decimal places**.

Even though there is randomness in the color composition in each bag, this list gives the likelihood that the machines have a systematic, meaningful, difference in how they blend the colors in each bag.

In [ ]:
# don't change this cell -- it is needed for the tests to work
q5_out = ordered_colors()
q5_colors = {'green', 'orange', 'purple', 'red', 'yellow'}
q5_test_colors = [x[0] for x in q5_out]

In [ ]:
grader.check("q5")

### Question 6 – Overall distributions 🏭

Now, suppose you would like to assess whether the two locations make similar amounts of each color overall. That is, suppose we:
* Combine and count up all the Skittles of each color that were made in Yorkville (e.g. 14303 total red skittles, 9091 total green skittles, etc.)
* Combine and count up all the Skittles of each color that were made in Waco.

**Are these distributions of colors similar?** Is the variation among the bags due to each factory making different amounts of each color?

Use a permutation test to assess whether the distribution of colors of Skittles made in Yorkville is statistically significantly different than those made in Waco. Set a significance level of 0.01 and determine whether you can reject a null hypothesis that answers the question above using a permutation test with 1000 trials. For your test statistic, use the **total variation distance (TVD)**.

Refer to [Lecture 10](https://dsc80.com/resources/lectures/lec10/lec10.html) to see an example of a [permutation test](https://www.inferentialthinking.com/chapters/12/Comparing_Two_Samples.html) that uses the [TVD](https://inferentialthinking.com/chapters/11/2/Multiple_Categories.html) as the test statistic. Some guidance:

- Our previous permutation tests have compared the mean number of (say) orange Skittles in Yorkville bags to the mean number of orange Skittles in Waco bags. The role of shuffling was to randomly assign bags to Yorkville and Waco.
- In this permutation test, we are **still** shuffling to randomly assign bags to Yorkville and Waco. The only difference is that after we randomly assign each bag to a factory, we will compute the distribution of colors amongst the two factories and find the TVD between those two distributions.

**Your job:** Create a function `same_color_distribution` that takes in no arguments and outputs a hard-coded **tuple** with the p-value and whether you `'Fail to Reject'` or `'Reject'` the null hypothesis.

In [ ]:
# don't change this cell -- it is needed for the tests to work
q6_out = same_color_distribution()

In [ ]:
grader.check("q6")

### Question 7 – Permutation testing vs. hypothesis testing 🧪

In each of the following scenarios, decide  whether  a  permutation test is appropriate to determine if there is a  significant difference between the quantities described. If a permutation test is appropriate, mark `'P'`. Otherwise, mark `'H'`.

Record your answers in the function `perm_vs_hyp` that outputs a list of length 5, containing the values `'P'` and `'H'`.

1. Compare the DSC 80 pass rate between second years and third years who take the class.
2. Compare the proportion of Data Science majors who have completed DSC 80 and the proportion of Data Science minors who have completed DSC 80.
3. Compare the proportion of students who have iPhones to the proportion of students who have Android phones (for simplicity, assume that all students either have an iPhone or an Android).
4. In DSC 80, we ask all students whether they liked DSC 40A or DSC 40B more. Compare the proportion of students who preferred DSC 40A to the proportion who preferred DSC 40B.
5. Compare the attendance rate of classes that play music before class vs. classes that do not play music before class.

***Hint:*** Think about the type of data you would collect in each case, and how you would simulate new data under the null hypothesis.

In [ ]:
# don't change this cell -- it is needed for the tests to work
q7_out = perm_vs_hyp()

In [ ]:
grader.check("q7")

## Part 4: Types of Missingness

First, let's recap the different mechanisms of missingness we studied in lecture.

### Missing by Design (MD)
- The missing field is deliberately missing. The missing field is deliberately set to null or not collected (hence, "missing by design").
- The missingness can be exactly predicted when a column will be null, with only knowledge of the other columns using a function of the rows of the dataset.

### Missing Completely at Random (MCAR)
- The missingness of missing value isn't related to the actual, unreported value itself, nor the values in any other fields. The missingness is not systematic.
- The missingness is unconditionally uniform across rows. MCAR doesn't bias the observed data.
- There is no relationship between the missing data and the any of the other data, observed or missing.

### Missing at Random (MAR)
- The missingness of the missing value has nothing to do with the value itself, but may be related to another field.
- The missingness is uniform across rows, perhaps conditional on another column. MAR biases the observed data, but is fixable.
- There is a systematic relationship between the missing values and the observed data (but not the missing values themselves).
- Difference between MD and MAR: If you can *exactly/always* determine missingness using the other columns, the missingness is MD. If there is just some sort of systematic relationship between the missing columns/values and other columns/values that may help us predict missingness, the missingness is MAR.

### Non-Ignorable (NI, aka NMAR)
- The missingness of the missing value is related to the actual, unreported value.
- NI biases the observed data in unobservable ways.
- There is relationship between the propensity of a value to be missing and its value.
- ***Note:*** In lecture, we referred to non-ignorable missingness as "not missing at random (NMAR)".

### Question 8 – After-purchase surveys 🛒

You run a small e-commerce website and send surveys out to customers after they purchase an item from your store. The survey asks whether the customer is satisfied with their purchase ("Yes" or "No"). Below, you are presented with possible datasets, each of which contains a column `'satisfied'` as described above, as well as a `'customer_id'` number corresponding to the customer and an `'item'` column describing the item that the customer purchased. **The column `'satisfied'` is missing data.**

For each of the following datasets, label the column `'satisfied'` as being `'MD'`, `'MCAR'`, `'MAR'`, or `'NI'`.

1. The dataset consists only of the columns `'customer_id'` and `'satisfied'`.
2. The dataset contains the `'customer_id'` of every customer with an account, even if they didn't make a purchase. Also, in this case, you notice everyone who was sent a survey filled it out.
3. The dataset contains a column specifying if the user later returned the item.
4. The dataset contains a column with the serial number for the item purchased.
5. The dataset contains a column with the price of the item purchased.

Record your answers in the function `after_purchase` that outputs a list of length 5, containing the values `'MD'`, `'MCAR'`, `'MAR'`, or `'NI'`. For some questions there may be multiple good answers, but there is generally one answer that is "best". If you are unsure, ask a tutor, but be prepared to provide justification for whichever answer(s) you think might be right.

***Disclaimer:*** We know that this lab has no hidden tests, and so it is possible to just look at the correct answers by running `grader.check`. This is not a good idea – you should really think about all of the questions here, since similar questions will be on the Midterm Exam.

In [ ]:
# don't change this cell -- it is needed for the tests to work
q8_out = after_purchase()

In [ ]:
grader.check("q8")

### Question 9 – Miscellaneous missingness questions 🕵️

In each of the following scenarios, choose the best answer out of the missingness types: `'MD'`, `'MCAR'`, `'MAR'`, and `'NI'`. Store your answers in a list of length 5, and have the function `multiple_choice` return that list.

1. UCSD has recently adopted GrubHub as the food pre-ordering app for campus restaurants, so you can order your food ahead of time and stop by before your next class. In a DataFrame of GrubHub app orders, which contains information such as `'restaurant'`, `'name'`, `'items'`, and `'total'`, the column `'delivery_address'` is often missing for UCSD students. Which is the most likely missingness mechanism for this column?


2. In a database of student records that records student profile data, such as `'name'`, `'home_address'`, `'ethnicity'`, etc., sometimes the `'middle_name'` column is missing. Which is the most likely missingness mechanism for this column?


3. The UCSD Club Basketball team creates a signup sheet for potential new members. The sheet contains the columns `'full_name'`, `'year'`, `'email'`, `'favorite_sports'`, `'number_of_sports_played'`, and `'sports_previously_played'`. The team president notices that many students left the `'sports_previously_played'` column blank. Which is the most likely missingness mechanism for this column?


4. After the 2022 Sun God Festival, Associated Students sends out a survey to all students about whether their expectations for the 2022 Sun God Festival were met, with all questions being optional. They notice that many students left the "Were you satisfied with the 2022 Sun God Festival?" question blank. Which is the most likely missingness mechanism for answers to this question?


5. UCSD has been using a two-factor authentication system, DUO, since October 16th, 2019. When using DUO, all UCSD accounts are assigned a unique code. UCSD's Service Desk, who maintains DUO, has a database that stores each user's code and their phone number, which users must provide when they sign up for DUO. They notice that many phone numbers are missing. Which is the most likely missingness mechanism for phone numbers?

In [ ]:
# don't change this cell -- it is needed for the tests to work
q9_out = multiple_choice()

In [ ]:
grader.check("q9")

## Congratulations! You're done! 🏁

Submit your `lab.py` file to Gradescope. Note that you only need to submit the `lab.py` file; this notebook should not be uploaded.

Before submitting, you should ensure that all of your work is in the `lab.py` file. You can do this by running the doctests below, which will verify that your work passes the public tests **and** that your work is in the `lab.py` file. Run the cell below; you should see no output.

In [ ]:
!python -m doctest lab.py

In addition, `grader.check_all()` will verify that your work passes the public tests.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()